# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
weather_csv = "weather_df.csv"

vacation_df = pd.read_csv(weather_csv)

vacation_df.head()


,Unnamed: 0,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude,Date
0,0,khatanga,RU,-7.44,99,100,6.62,71.9667,102.5000,1677963827
1,1,albany,US,37.38,89,100,4.09,42.6001,-73.9662,1677963574
2,2,rawson,AR,80.65,18,56,5.66,-43.3002,-65.1023,1677963828
3,3,sobolevo,RU,21.07,88,32,13.33,54.4299,31.8996,1677963828
4,4,nikolskoye,RU,31.89,98,98,4.88,59.7035,30.7861,1677963829


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
#Configure gmaps
gmaps.configure(api_key=g_key)

# Humidity as the weight
weights = vacation_df["Humidity"]

# Lat & Lng locations 
locations = vacation_df[["Latitude", "Longitude"]]

# Create map
fig = gmaps.figure(display_toolbar=True)

#Add heatmap layer
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights,max_intensity=max(vacation_df['Humidity'])))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
# Narrow down the cities to fit weather conditions.
weather_conditions = vacation_df[(vacation_df["Temperature"] >70) & (vacation_df["Temperature"] <80) \
                            & (vacation_df["Windspeed"] <10) & (vacation_df["Cloudiness"] ==0)]\
                              .dropna()

# Drop any rows will null values.

weather_conditions

,Unnamed: 0,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude,Date
134,134,kodinar,IN,78.51,49,0,6.29,20.7903,70.7031,1677963912
241,241,pisco,PE,78.85,65,0,6.91,-13.7000,-76.2167,1677963981
254,254,durango,MX,76.26,5,0,5.79,24.8333,-104.8333,1677963782
321,321,cabo san lucas,MX,76.60,56,0,3.44,22.8909,-109.9124,1677963987
342,342,broken hill,AU,71.40,32,0,1.81,-31.9500,141.4333,1677964045
374,374,sharjah,AE,73.38,73,0,4.61,25.3573,55.4033,1677963942
398,398,bellary,IN,73.31,42,0,5.79,15.1500,76.9333,1677964078
410,410,minab,IR,72.00,75,0,2.57,27.1467,57.0801,1677964088
428,428,asyut,EG,78.51,10,0,6.91,27.1810,31.1837,1677964104
458,458,bhuban,IN,70.92,47,0,1.30,20.8833,85.8333,1677964125


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
hotel_df = weather_conditions.loc[:,["City", "Country", "Latitude", "Longitude"]]

hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Latitude,Longitude,Hotel Name
134,kodinar,IN,20.7903,70.7031,
241,pisco,PE,-13.7000,-76.2167,
254,durango,MX,24.8333,-104.8333,
321,cabo san lucas,MX,22.8909,-109.9124,
342,broken hill,AU,-31.9500,141.4333,
374,sharjah,AE,25.3573,55.4033,
398,bellary,IN,15.1500,76.9333,
410,minab,IR,27.1467,57.0801,
428,asyut,EG,27.1810,31.1837,
458,bhuban,IN,20.8833,85.8333,


In [19]:
import json
import pprint

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}


for index, row in hotel_df.iterrows(): 
   # print(row)
    #break 
    
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{latitude},{longitude}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    pprint.pprint(response)
    
    # extract results
    results = response['results']
    
   #print(json.dumps(response, indent=4, sort_keys=True))
#     break

# save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    except (KeyError, IndexError):
        print("No hotel")

Retrieving Results for Index 134: kodinar.
{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 20.7908388,
                                        'lng': 70.70110939999999},
                           'viewport': {'northeast': {'lat': 20.79221277989272,
                                                      'lng': 70.70240422989272},
                                        'southwest': {'lat': 20.78951312010728,
                                                      'lng': 70.69970457010729}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'icon_background_color': '#909CE1',
              'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
              'name': 'Hotel Sun View Regency Kodinar',
              'opening_hours': {'open_now': True},
              'photos': [{'height': 1226,
                          'ht

{'html_attributions': [],
 'next_page_token': 'AfLeUgMsECWHIRZReRRPk85haEYEreHPH1wrTz0KxiF0NoaCioPRz5ZhjO8FZ22skVOJupqY1mILiF3gLBEf7YPWhwzbv7TERKNPDnj956JMyARPw67ZTT8fTz5pZbSPZesVyVDMSQPgT17VV0-n4xI0iKxZQN9k4u4jimtOocIvuDnBptjzuHXGF_IgPbHouOxHKVLYdUvXbbrXJAQTquz0f1LR0LsVQwP-TQOnfzY4XLEkLCNzIVR4AxX-uauITgQkSGdHADlxEc49JL5BE3f9qz9I6-F2spik1w2JgEvPaq8SIoFvEnWITAIo7oKSmH-DvkLOdF9HzGL8kZDTqqXp6v8WVqHRnD5KBcARYCnxNcVZkM-0r5oW_duAB9jkPUP3DVkpIbL9KrrqCwMiBdPEy9Xnu7xdLj3cV7MsaMn487h5R-ET4IZYrdBfeRaxgBI',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': -13.7078382, 'lng': -76.1990199},
                           'viewport': {'northeast': {'lat': -13.70648892010728,
                                                      'lng': -76.19763412010728},
                                        'southwest': {'lat': -13.70918857989272,
                                                      'lng': -76.20033377989273}}},
              'icon': 'https://maps.gstati

{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
No hotel
Retrieving Results for Index 321: cabo san lucas.
{'error_message': 'You must enable Billing on the Google Cloud Project at '
                  'https://console.cloud.google.com/project/_/billing/enable '
                  'Learn more at '
                  'https://developers.google.com/maps/gmp-get-started',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
No hotel
Retrieving Results for Index 342: broken hill.
{'html_attributions': [],
 'next_page_token': 'AfLeUgONARQ8UzyK6w9aOAZrfxI9Ajh2EXKy0LbB9bJP2x5EifGUN1uiEkcsb0bM5Z3abAvpcqCz3KwQFgIbQGUUt2teKYcmI9nOHIdNbM8Xj_Fz4tCe04enuXdlHgmTSSK8K3wxAlz3Gqr0u87xs-9nDhEuQPRw_D_EOrs4GHyKno4K6PLA78Elljfo6qs6eb_F0O5UWAVBY-pu4a0yITesffbTF9W6f5Xrq7Rp5OHWmyPh49QFzwox0A7b8JYpi9v9s9rnyAM-EWpfed0yhEqGxvWiAwV41vTvOt6ODa0QfQI2uPAIjUE4Y7IWvkgGCrsOJkPmQG-JQoppJGv5eYTW5l5pJRCTDk_tvGUFUjAbGkESq2QQnez0icFcjtbRGgXqxbQFfqq9LbI3MXqiQwcVCOCtgQ82b3AwrjtVcFE8AkLHL_ZC-o

{'html_attributions': [],
 'next_page_token': 'AfLeUgO3MHKXiG89yW8igpD6IWNFkX21oJubGuzH_bGF0Fqrj71LqiHkIr0d100OLJytDB76HtDTGI5koYDMra-5Y6WV0Gd-n1egGI8gdDBTFTfDbPAOJjAEByZSUrcilk1BiY7xlXzUQLLZR_O16RLJs4eyU6zxREZ3p_ofTMcTqSOoVWbAHukr-k6ODbDOO4ZytZY8l6kE1PSSnU4JuuzHUlhs7m6QNWbMDE31XIlrGNGMcNkDhcifHyCc3N_NotPoW5u6VRBf1vDRmnUrh6oLiK0lefXSzcrrZM8UVE2GJ29utvbpAYHN1flawXRHOySS0V-fCFORDt8kda5GC_KeMpsnRgBQLbSEHepd3_QwgZC6EsAmF90eLE9iQYwv45TyR9fQyV7glegtqEXp_k2lPvAT0olg4trYGDDVF98qCMv1kuGpcCA2O4lFOzZp8D0',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 25.3446233, 'lng': 55.3888573},
                           'viewport': {'northeast': {'lat': 25.34598667989272,
                                                      'lng': 55.39026487989272},
                                        'southwest': {'lat': 25.34328702010728,
                                                      'lng': 55.38756522010728}}},
              'icon': 'https://maps.gstatic.com/

{'html_attributions': [],
 'next_page_token': 'AfLeUgPTbNN02ZjGavyavzc94U8B-IfitmF-dgHB-Z5jh1ZzdMZ8QJyT16XHi5hbNU9mmqO5cBGXrubDjP8Z2W_6z9_bfnZIDlBEYxehJgTv_MvFEJwvz8Cje8cZL9RZ6qaGoTctSSwzv9De5QMCSs2CHeaaDv3xegRwCjNvXFJWLzTHMsqV0ZStT6DqVZz6sc3POTZ0Q4hyjTvw3qjcg4bWec8Q6WenHzFgQAX_kNJ5JCTGTLULjWfZRPw8n1xBpQtcviFaK81a0eihX1cSyF6Woza5SC_t1IQu57V1LFYCG33lJJ9miwO8oSpPnK4Fw2ULgcccaBywkCP84iR3DQYpVOwPC2teZrR5uK6lidcG_J0gn1P4HGAAz249naKUTDWaxYO-6791zlQ-S99fuXijIGbKwkRlVm1ChA6jlclrqRRkFzLzYOqtvY9ZbVTyVNo',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 15.1445384, 'lng': 76.9266902},
                           'viewport': {'northeast': {'lat': 15.14570152989273,
                                                      'lng': 76.92808162989272},
                                        'southwest': {'lat': 15.14300187010728,
                                                      'lng': 76.92538197010727}}},
              'icon': 'https://maps.gstatic.com/

{'error_message': 'You must enable Billing on the Google Cloud Project at '
                  'https://console.cloud.google.com/project/_/billing/enable '
                  'Learn more at '
                  'https://developers.google.com/maps/gmp-get-started',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
No hotel
Retrieving Results for Index 458: bhuban.
{'error_message': 'You must enable Billing on the Google Cloud Project at '
                  'https://console.cloud.google.com/project/_/billing/enable '
                  'Learn more at '
                  'https://developers.google.com/maps/gmp-get-started',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}
No hotel
Retrieving Results for Index 480: lamar.
{'error_message': 'You must enable Billing on the Google Cloud Project at '
                  'https://console.cloud.google.com/project/_/billing/enable '
                  'Learn more at '
                  'https://developers.googl

In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [22]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))